In [6]:
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np

In [7]:
import tensorflow as tf

import keras.models 

from keras.models import Model
from keras.layers import Dense

import keras_metrics

Using TensorFlow backend.


In [8]:
train_data = pd.read_csv("../final_train_companies.csv")
train_data.head()

,Unnamed: 0,category,suspicious,turnover,transaction_count,io_ratio,inactive_days_average,inactive_days_max,n_of_accounts,distinct_counterparties,...,50,66,94,113,123,134,141,172,176,191
0,1,0,0,-0.117401,-0.469817,0.5,0.687737,-0.450931,0.996488,-0.321589,...,0,0,0,0,0,0,0,0,0,0
1,23,0,0,-0.117224,-0.486643,1.0,-0.036124,-0.385296,-0.002259,-0.321589,...,0,0,1,0,0,0,0,0,0,0
2,31,1,0,-0.113700,-0.486643,0.0,-0.580980,-1.304184,0.996488,-0.321589,...,0,0,0,0,0,0,0,1,0,0
3,32,1,0,0.005481,-0.469817,0.5,-0.211600,-0.201519,-0.002259,-0.321589,...,0,0,0,0,0,0,0,0,0,0
4,33,0,0,-0.119004,-0.469817,0.0,-0.418837,-0.897248,-1.001005,-0.321589,...,0,0,1,0,0,0,0,0,0,0


In [36]:
test_data = pd.read_csv("../final_test_companies.csv").reset_index()
original_test = pd.read_csv("../test.csv")
test_data["customer"] = original_test[original_test["category"] != 0].reset_index()["customer"]
test_data.head(100)

,index,Unnamed: 0,category,turnover,transaction_count,io_ratio,inactive_days_average,inactive_days_max,n_of_accounts,distinct_counterparties,...,66,94,113,123,134,141,172,176,191,customer
0,0,1,0,-0.118999,-0.452992,0.000000,0.507164,0.336686,-1.001005,-0.321589,...,0,0,0,0,0,0,1,0,0,91000001
1,1,9,0,-0.119041,-0.486643,1.000000,0.594216,0.454829,-1.001005,-0.321589,...,0,1,0,0,0,0,0,0,0,91000009
2,2,10,0,-0.079890,0.438739,0.410714,-0.799982,-0.595328,0.996488,-0.263997,...,0,0,0,0,0,0,0,0,0,91000010
3,3,20,0,-0.106234,-0.352042,0.333333,-0.575883,-0.004615,0.996488,-0.321589,...,0,0,0,0,0,0,0,0,0,91000020
4,4,22,0,-0.105683,0.253663,0.377778,-0.772729,-0.319662,-0.002259,0.715062,...,0,1,0,0,0,0,0,0,0,91000022
5,5,34,0,-0.115543,-0.486643,1.000000,0.135626,-0.162138,0.996488,-0.321589,...,0,0,0,0,0,0,0,0,0,91000034
6,6,40,0,-0.117675,-0.486643,0.000000,-0.491576,-1.238549,-1.001005,-0.321589,...,0,0,0,0,1,0,0,0,0,91000040
7,7,43,0,-0.118983,-0.469817,0.500000,0.618331,1.465605,0.996488,-0.321589,...,0,0,0,0,0,0,0,0,0,91000043
8,8,50,0,-0.093668,0.220012,0.325581,-0.773906,-0.188392,-0.002259,-0.206405,...,0,0,0,0,0,0,0,0,0,91000050
9,9,51,1,1.055553,1.448246,0.379310,-0.822137,-0.240900,-1.001005,6.301459,...,0,0,0,0,0,0,1,0,0,91000051


In [10]:
train_features = train_data.drop(["Unnamed: 0", "suspicious"], axis=1).values
train_labels = train_data["suspicious"].values

In [11]:
def cost(preds, labels, tresh):
    investigation_cost = sum(preds > tresh) * 1000
    fine_cost = sum(labels[preds < tresh]) * 60000
    return investigation_cost + fine_cost

In [12]:
full_train_features = train_features
full_train_labels = train_labels

In [13]:
full_augmented_train_features = full_train_features
full_augmented_train_labels = full_train_labels
for i in range(100):
    full_augmented_train_features = np.concatenate((full_augmented_train_features, full_train_features[full_train_labels == 1]))
    full_augmented_train_labels = np.concatenate((full_augmented_train_labels, np.array([1] * full_train_features[full_train_labels == 1].shape[0])))

In [14]:
full_augmented_train_labels.sum()/len(full_augmented_train_labels)

0.3281901274767441

In [15]:
encoding_dim = train_features.shape[-1]
model_input = keras.models.Input(shape=(encoding_dim,))
x = Dense(32, activation='tanh')(model_input)
x = keras.layers.Dropout(0.1)(x)
x = Dense(16, activation='tanh')(x)
x = keras.layers.Dropout(0.1)(x)
#x = Dense(8, activation='tanh')(x)
#x = keras.layers.Dropout(0.1)(x)
out = Dense(2, activation='sigmoid')(x)

model = Model(model_input, out)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[keras_metrics.precision(), keras_metrics.recall()])

In [16]:
for i in range(10):
    history = model.fit(full_augmented_train_features, tf.keras.utils.to_categorical(full_augmented_train_labels, 2), epochs=1, batch_size=256, shuffle=True,
                verbose=1, )   

Epoch 1/1
296054/296054 [==============================] - 3s 9us/step - loss: 0.5049 - precision: 0.6832 - recall: 0.4552
Epoch 1/1
296054/296054 [==============================] - 3s 9us/step - loss: 0.4646 - precision: 0.7197 - recall: 0.5277
Epoch 1/1
296054/296054 [==============================] - 3s 9us/step - loss: 0.4558 - precision: 0.7221 - recall: 0.5444
Epoch 1/1
296054/296054 [==============================] - 3s 9us/step - loss: 0.4490 - precision: 0.7267 - recall: 0.5552
Epoch 1/1
296054/296054 [==============================] - 3s 9us/step - loss: 0.4441 - precision: 0.7283 - recall: 0.5646
Epoch 1/1
296054/296054 [==============================] - 3s 9us/step - loss: 0.4395 - precision: 0.7298 - recall: 0.5735
Epoch 1/1
296054/296054 [==============================] - 3s 9us/step - loss: 0.4350 - precision: 0.7326 - recall: 0.5804
Epoch 1/1
296054/296054 [==============================] - 3s 9us/step - loss: 0.4312 - precision: 0.7316 - recall: 0.5884
Epoch 1/1
296054

In [17]:
val = model.predict(train_data.drop(["Unnamed: 0","suspicious"], axis=1).values)[:,1]

In [18]:
val.mean()

0.20293064

In [19]:
val.std()

0.19659862

In [37]:
test_data["predictions"] = model.predict(test_data.drop(["Unnamed: 0","customer","index"], axis=1).values)[:,1]

In [38]:
len(test_data[test_data["predictions"] >= 0.5])

19439

In [26]:
#test_data[test_data["predictions"] >= 0.5]

In [28]:
len(test_data[test_data["predictions"] >= 0.5].reset_index()[["customer"]])

19439

In [29]:
test_data[test_data["predictions"] >= 0.5].reset_index()[["customer"]].to_csv("../comp_nr_nn.csv", index=False)

In [30]:
final_df = pd.concat((pd.read_csv("../cust_nr_nn.csv"), pd.read_csv("../comp_nr_nn.csv")), axis=0)

In [31]:
final_df.reset_index()[["customer"]].to_csv("../all.csv", index=False)

In [32]:
len(final_df)

105868

In [42]:
test_data[["customer","predictions"]].to_csv("../test_predictions_companies_prob_nn.csv", index=False)

In [41]:
len(test_data)

199805

In [ ]:
val

In [56]:
visu = test_data.sort_values("predictions", ascending=False)
visu.head()

,index,Unnamed: 0,category,turnover,transaction_count,io_ratio,inactive_days_average,inactive_days_max,n_of_accounts,distinct_counterparties,...,94,113,123,134,141,172,176,191,customer,predictions
67337,67337,337914,1,-0.097638,5.822776,0.523936,-0.841155,-0.162138,0.996488,0.657470,...,0,0,0,1,0,0,0,0,91337914,0.995534
173901,173901,871757,0,-0.115056,1.414596,0.622807,-0.822529,-0.490312,-0.002259,-0.148814,...,0,0,0,0,0,0,0,0,91871757,0.995472
45588,45588,227786,0,-0.115798,1.717448,0.583333,-0.823509,-0.201519,-1.001005,1.233388,...,0,0,0,0,0,0,0,0,91227786,0.995331
89264,89264,448228,0,0.253749,3.770112,0.523622,-0.837234,-0.227773,-1.001005,2.039672,...,0,0,0,0,0,1,0,0,91448228,0.995297
169710,169710,850937,1,-0.064537,8.800821,0.502712,-0.844488,-0.345916,-1.001005,11.081572,...,0,0,0,0,1,0,0,0,91850937,0.995059


In [57]:
visu = visu.drop(["index","Unnamed: 0","predictions","customer"], axis=1)
visu.head()

,category,turnover,transaction_count,io_ratio,inactive_days_average,inactive_days_max,n_of_accounts,distinct_counterparties,channel_risk,atm_withdrawal,...,50,66,94,113,123,134,141,172,176,191
67337,1,-0.097638,5.822776,0.523936,-0.841155,-0.162138,0.996488,0.657470,0.143886,-0.030537,...,0,0,0,0,0,1,0,0,0,0
173901,0,-0.115056,1.414596,0.622807,-0.822529,-0.490312,-0.002259,-0.148814,0.249153,-0.032430,...,1,0,0,0,0,0,0,0,0,0
45588,0,-0.115798,1.717448,0.583333,-0.823509,-0.201519,-1.001005,1.233388,0.131561,-0.032162,...,0,0,0,0,0,0,0,0,0,0
89264,0,0.253749,3.770112,0.523622,-0.837234,-0.227773,-1.001005,2.039672,0.501804,-0.000804,...,0,0,0,0,0,0,0,1,0,0
169710,1,-0.064537,8.800821,0.502712,-0.844488,-0.345916,-1.001005,11.081572,0.205006,-0.018550,...,0,0,0,0,0,0,1,0,0,0


In [60]:
visu.head(1000).to_csv("../visu.csv", index=False)

In [64]:
train_data[train_data["suspicious"] == 0].head(1000).drop(["Unnamed: 0", "suspicious"], axis=1).to_csv("../visu_non_suspicious.csv", index=False)

In [65]:
train_data[train_data["suspicious"] == 1].head(1000).drop(["Unnamed: 0", "suspicious"], axis=1).to_csv("../visu_suspicious.csv", index=False)